### Checkpoint 19.8: Challenge
The scenario
The housing market is one of the most crucial parts of the economy for every country. Purchasing a home is one of the primary ways to build wealth and savings for people. In this respect, predicting prices in the housing market is a very central topic in economic and financial circles.

The house price dataset from Kaggle includes several features of the houses along with their sale prices at the time they are sold. So far, in this module, you built and implemented some models using this dataset.

In this challenge, you are required to improve your model with respect to its prediction performance.

To complete this challenge, submit a Jupyter notebook containing your solutions to the following tasks.

**Steps:**

1. Load the houseprices data from Thinkful's database.


2. Do data cleaning, exploratory data analysis, and feature engineering. You can use your previous work in this module. But make sure that your work is satisfactory.


3. Now, split your data into train and test sets where 20% of the data resides in the test set.


4. Build several linear regression models including Lasso, Ridge, or ElasticNet and train them in the training set. Use k-fold cross-validation to select the best hyperparameters if your models include one!

5. Evaluate your best model on the test set.

6. So far, you have only used the features in the dataset. However, house prices can be affected by many factors like economic activity and the interest rates at the time they are sold. So, try to find some useful factors that are not included in the dataset. Integrate these factors into your model and assess the prediction performance of your model. Discuss the implications of adding these external variables into your model.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import os

In [2]:
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))


housedf = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

In [3]:
# Creating the for loop to take care of all missing values in object columns.
column = (['poolqc', 'fence', 'miscfeature', 'alley', 'fireplacequ', 'garagecond', 'garagetype', 'garagefinish', 'garagequal', 
            'bsmtexposure', 'bsmtfintype2', 'bsmtfintype1', 'bsmtcond', 'bsmtqual', 'masvnrtype', 'electrical'])

for col_name in column:
   housedf[col_name].fillna('blank', inplace= True)

In [4]:
  # Since this feature has a min. value of 0 it won't hurt if I turn all missing values to a 0 as well
housedf['masvnrarea'].fillna(0, inplace= True)

# Since the remaining two features only have a minimal number of null values and can't be set to 0 I'm going to set them to 
# the average of the column.

housedf['lotfrontage'].fillna(housedf['lotfrontage'].mean(),inplace=True)
housedf['garageyrblt'].fillna(housedf['garageyrblt'].mean(),inplace=True)

In [18]:
non_numeric_columns = housedf.select_dtypes(['object']).columns
numeric_columns = housedf.select_dtypes(['int64', 'float64']).columns

In [19]:
# Adding new feature to our model
X = housedf[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf']] 
Y = housedf.saleprice

# Creating a new feature for the interaction of overall quality and number of cars
housedf['quality_and_cars'] = housedf['overallqual'] * housedf['garagecars']

In [20]:
# Adding in important packages for K-folds cross-val.
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn import metrics

In [17]:
# OLS Regression- 1 fold
X = housedf[['overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf', 'quality_and_cars']] 
Y = housedf.saleprice

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

print('The number of total observations in the dataset is {}'.format(X.shape[0]))
print('The number of observations in the training set is {}'.format(X_train.shape[0]))
print('The  number of observations in the test set is {}'.format(X_test.shape[0]))
print('\n')

# Fitting an OLS model using SkLearn instead of my go-to
lrm= LinearRegression()
lrm.fit(X_train, Y_train)

# Making predictions
Y_preds_train = lrm.predict(X_train)
Y_preds_test= lrm.predict(X_test)


# OLS- 5 fold cross-val.
scores = cross_val_score(lrm, X, Y, cv=5)
kf_shuffle = KFold(n_splits=5, shuffle=True)
second_scores = cross_val_score(lrm, X_train, Y_train, cv = kf_shuffle)


print("Mean of scores:", np.mean(scores))
print("STD of scores:", np.std(scores))      
print("\n")
print("Mean of second scores:", np.mean(second_scores))
print("STD of second scores:", np.std(second_scores)) 

The number of total observations in the dataset is 1460
The number of observations in the training set is 1168
The  number of observations in the test set is 292


Mean of scores: 0.7897659496817377
STD of scores: 0.03967784479867933


Mean of second scores: 0.7805439925928985
STD of second scores: 0.0466969236210171


In [21]:
# Ridge regression model- 1 fold
from sklearn.linear_model import Ridge

ridgeregr = Ridge()
ridgeregr.fit(X_train, Y_train)

# Time to make predictions!
Y_preds_train = ridgeregr.predict(X_train)
Y_preds_test = ridgeregr.predict(X_test)

# Ridge Regression- 5 fold cross-val.
ridge_scores = cross_val_score(ridgeregr, X, Y, cv=5)
kf_shuffle = KFold(n_splits=5, shuffle=True)
ridge_second_scores = cross_val_score(ridgeregr, X_train, Y_train, cv = kf_shuffle)


print("Mean of scores:", np.mean(ridge_scores))
print("STD of scores:", np.std(ridge_scores))      
print("\n")
print("Mean of second scores:", np.mean(ridge_second_scores))
print("STD of second scores:", np.std(ridge_second_scores)) 

Mean of scores: 0.749454686714951
STD of scores: 0.059536111922219456


Mean of second scores: 0.7820881590473936
STD of second scores: 0.05364369580099606


In [23]:
# LASSO Regression - 1 fold
from sklearn.linear_model import Lasso

lassoregr = Lasso()
lassoregr.fit(X_train, Y_train)

# Time to make predictions!
Y_preds_train = lassoregr.predict(X_train)
Y_preds_test = lassoregr.predict(X_test)

# LASSO Regression- 5 fold cross-val.
lasso_scores = cross_val_score(lassoregr, X, Y, cv=5)
kf_shuffle = KFold(n_splits=5, shuffle=True)
lasso_second_scores = cross_val_score(lassoregr, X_train, Y_train, cv = kf_shuffle)


print("Mean of scores:", np.mean(lasso_scores))
print("STD of scores:", np.std(lasso_scores))      
print("\n")
print("Mean of second scores:", np.mean(lasso_second_scores))
print("STD of second scores:", np.std(lasso_second_scores)) 


Mean of scores: 0.7494337062134336
STD of scores: 0.05946957317527055


Mean of second scores: 0.7879472886589733
STD of second scores: 0.07414628423837742


In [24]:
#ElasticNet Regression - 1 fold
from sklearn.linear_model import ElasticNet

elasticregr = ElasticNet( l1_ratio=0.5)
elasticregr.fit(X_train, Y_train)

# Time to make predictions!
Y_preds_train = elasticregr.predict(X_train)
Y_preds_test = elasticregr.predict(X_test)

# ElasticNet Regression- 5 fold cross-val.
en_scores = cross_val_score(elasticregr, X, Y, cv=5)
kf_shuffle = KFold(n_splits=5, shuffle=True)
en_second_scores = cross_val_score(elasticregr, X_train, Y_train, cv = kf_shuffle)


print("Mean of scores:", np.mean(en_scores))
print("STD of scores:", np.std(en_scores))      
print("\n")
print("Mean of second scores:", np.mean(en_second_scores))
print("STD of second scores:", np.std(en_second_scores)) 

Mean of scores: 0.7382654711451218
STD of scores: 0.07953636494016218


Mean of second scores: 0.769354258332162
STD of second scores: 0.06017023900053124


### Break-Down of Model Success

In [34]:
print('The mean of scores in the OLS model after K-Fold of 5:', round(np.mean(second_scores), 4))
print('The STD of scores in the OLS model after K-Fold of 5:', round(np.std(second_scores), 4))
print('\n')
print('The mean of scores in the Ridge Model after K-Fold of 5:', round(np.mean(ridge_second_scores), 4))
print('The STD of scores in the Ridge model after K-Fold of 5:', round(np.std(ridge_second_scores), 4))
print('\n')
print('The mean of scores in the LASSO Model after K-Fold of 5:', round(np.mean(lasso_second_scores), 4))
print('The STD of scores in the LASSO model after K-Fold of 5:', round(np.std(lasso_second_scores), 4))
print('\n')
print('The mean of scores in the ElasticNet Model after K-Fold of 5:', round(np.mean(en_second_scores), 4))
print('The STD of scores in the ElasticNet model after K-Fold of 5:', round(np.std(en_second_scores), 4))
print('\n')

The mean of scores in the OLS model after K-Fold of 5: 0.7805
The STD of scores in the OLS model after K-Fold of 5: 0.0467


The mean of scores in the Ridge Model after K-Fold of 5: 0.7821
The STD of scores in the Ridge model after K-Fold of 5: 0.0536


The mean of scores in the LASSO Model after K-Fold of 5: 0.7879
The STD of scores in the LASSO model after K-Fold of 5: 0.0741


The mean of scores in the ElasticNet Model after K-Fold of 5: 0.7694
The STD of scores in the ElasticNet model after K-Fold of 5: 0.0602




After looking at the statistics above I noticed the following thing:
* Each type of regression was pretty close in mean scores after running with a K-folds of 5
* Although the highest mean score was for the LASSO model (0.7879) it also had the highest STD, which makes me suspicious of overfitting
* The Ridge regression model has a slightly higher mean of scores than the OLS model and a slightly higher STD, but it still seems reasonable, so I am planning on using this type of regression on my test data.

In [36]:
# Trying out my Ridge regression on my test data
ridgeregr.fit(X_test, Y_test)

final_scores = cross_val_score(ridgeregr, X, Y, cv=5)
kf_shuffle = KFold(n_splits=5, shuffle=True)
final2_scores = cross_val_score(ridgeregr, X_test, Y_test, cv = kf_shuffle)


print("Mean of scores:", np.mean(final_scores))
print("STD of scores:", np.std(final_scores))      
print("\n")
print("Mean of second scores:", np.mean(final2_scores))
print("STD of second scores:", np.std(final2_scores)) 

Mean of scores: 0.749454686714951
STD of scores: 0.059536111922219456


Mean of second scores: 0.7926615561837503
STD of second scores: 0.059573458446894365


Overall, I am pleased with the performance of the Ridge regression on my data set. The statistics on mean of scores and STD for both training and test data is pretty consistant and I feel like this is a great starting point for my model. If I was using this model for work or another assignment that I would have the time to tinker on and improve I would keep engineering my features to see if I could get a higer mean of scores in each iteration!

## Step 6: Outside Factors
In this step we were asked to look at other factors, outside of this database, that would make the most sense to add. Several factors that influence house sale prices that I considered were mortgage rates, income per capita of the city/neighborhood for the year of sale, houses on the market at time of sale, unemployement numbers for the year of sale, and crime rates for the neighborhood of the houses. Since these data are not included in the original dataset I had to do some digging to see what information I could find to bring in to this project. I was able to locate information for interest rates on 30 year mortgages for each month in the time frame series. I have joined the datasets below, and will re-run my model with the added feature of interest rate to see if that made any difference.

In [39]:
os.getcwd()

'C:\\Users\\gothv\\Practice\\Thinkful- Assignments'

In [42]:
os.chdir('C:\\Users\\gothv\\OneDrive\\Desktop\\Data\\Datasets')

In [43]:
os.getcwd()

'C:\\Users\\gothv\\OneDrive\\Desktop\\Data\\Datasets'

In [48]:
interestdf = pd.read_csv('interest_rates.csv')

In [49]:
interestdf.head()

,mosold,yrsold,date,Interest Rate
0,1,2006,5-Jun,6.29%
1,2,2006,6-Feb,6.41%
2,3,2006,6-Mar,6.48%
3,4,2006,6-Apr,6.63%
4,5,2006,6-May,6.75%


In [47]:
housedf.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice,quality_and_cars
0,1,60,RL,65.0,8450,Pave,blank,Reg,Lvl,AllPub,...,blank,blank,blank,0,2,2008,WD,Normal,208500,14
1,2,20,RL,80.0,9600,Pave,blank,Reg,Lvl,AllPub,...,blank,blank,blank,0,5,2007,WD,Normal,181500,12
2,3,60,RL,68.0,11250,Pave,blank,IR1,Lvl,AllPub,...,blank,blank,blank,0,9,2008,WD,Normal,223500,14
3,4,70,RL,60.0,9550,Pave,blank,IR1,Lvl,AllPub,...,blank,blank,blank,0,2,2006,WD,Abnorml,140000,21
4,5,60,RL,84.0,14260,Pave,blank,IR1,Lvl,AllPub,...,blank,blank,blank,0,12,2008,WD,Normal,250000,24


In [56]:
# Joining my interest rate df to my house df to see if interest rate factors into this model.
left= housedf
right= interestdf

result = pd.merge(left, right, on= ['mosold', 'yrsold'])

In [58]:
result.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice,quality_and_cars,date,Interest Rate
0,1,60,RL,65.0,8450,Pave,blank,Reg,Lvl,AllPub,...,blank,0,2,2008,WD,Normal,208500,14,8-Feb,6.41%
1,11,20,RL,70.0,11200,Pave,blank,Reg,Lvl,AllPub,...,blank,0,2,2008,WD,Normal,129500,5,8-Feb,6.41%
2,202,20,RL,75.0,10125,Pave,blank,Reg,Lvl,AllPub,...,blank,0,2,2008,WD,Normal,171500,12,8-Feb,6.41%
3,269,30,RM,71.0,6900,Pave,blank,Reg,Lvl,AllPub,...,blank,0,2,2008,WD,Normal,120500,5,8-Feb,6.41%
4,423,20,RL,100.0,21750,Pave,blank,Reg,HLS,AllPub,...,blank,0,2,2008,WD,Normal,113000,10,8-Feb,6.41%


In [64]:
result.rename(columns= {"Interest Rate": 'interest_rate'}, inplace = True)

In [75]:
result['interest_rate'] = result.interest_rate.str.replace('%', '')

In [76]:
result['interest_rate'] = result['interest_rate'].astype(float)

In [79]:
result.corr()

,id,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,...,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,quality_and_cars,interest_rate
id,1.000000,0.011156,-0.009601,-0.033226,-0.028365,0.012609,-0.012713,-0.021998,-0.051071,-0.005024,...,0.002889,-0.046635,0.001330,0.057044,-0.006242,0.021172,0.000712,-0.021917,-0.007499,-0.008740
mssubclass,0.011156,1.000000,-0.357056,-0.139781,0.032628,-0.059316,0.027850,0.040581,0.023573,-0.069836,...,-0.012037,-0.043825,-0.026030,0.008283,-0.007683,-0.013585,-0.021407,-0.084284,-0.035786,0.022793
lotfrontage,-0.009601,-0.357056,1.000000,0.306795,0.234196,-0.052820,0.117598,0.082746,0.178699,0.215828,...,0.009790,0.062335,0.037684,0.180868,0.001168,0.010158,0.006768,0.334901,0.296171,-0.011702
lotarea,-0.033226,-0.139781,0.306795,1.000000,0.105806,-0.005636,0.014228,0.013788,0.103321,0.214103,...,-0.018340,0.020423,0.043160,0.077672,0.038068,0.001205,-0.014261,0.263843,0.149685,0.022744
overallqual,-0.028365,0.032628,0.234196,0.105806,1.000000,-0.091932,0.572323,0.550684,0.407252,0.239666,...,-0.113937,0.030371,0.064886,0.065166,-0.031406,0.070815,-0.027347,0.790982,0.832345,0.023745
overallcond,0.012609,-0.059316,-0.052820,-0.005636,-0.091932,1.000000,-0.375983,0.073741,-0.125694,-0.046231,...,0.070356,0.025504,0.054811,-0.001985,0.068777,-0.003511,0.043950,-0.077856,-0.190881,0.030426
yearbuilt,-0.012713,0.027850,0.117598,0.014228,0.572323,-0.375983,1.000000,0.592855,0.311600,0.249503,...,-0.387268,0.031355,-0.050364,0.004950,-0.034383,0.012398,-0.013618,0.522897,0.611622,0.003227
yearremodadd,-0.021998,0.040581,0.082746,0.013788,0.550684,0.073741,0.592855,1.000000,0.176529,0.128451,...,-0.193919,0.045286,-0.038740,0.005829,-0.010286,0.021490,0.035743,0.507101,0.525460,-0.003781
masvnrarea,-0.051071,0.023573,0.178699,0.103321,0.407252,-0.125694,0.311600,0.176529,1.000000,0.261256,...,-0.109907,0.019144,0.062248,0.011928,-0.029512,-0.006723,-0.008317,0.472614,0.448035,0.028430
bsmtfinsf1,-0.005024,-0.069836,0.215828,0.214103,0.239666,-0.046231,0.249503,0.128451,0.261256,1.000000,...,-0.102303,0.026451,0.062021,0.140491,0.003571,-0.015727,0.014359,0.386420,0.266566,-0.019337


In [77]:
X = result[['overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf', 'quality_and_cars', 'interest_rate']] 
Y = result.saleprice

ridgeregr = Ridge()
ridgeregr.fit(X_train, Y_train)

# Time to make predictions!
Y_preds_train = ridgeregr.predict(X_train)
Y_preds_test = ridgeregr.predict(X_test)

# Ridge Regression- 5 fold cross-val.
ridge_scores = cross_val_score(ridgeregr, X, Y, cv=5)
kf_shuffle = KFold(n_splits=5, shuffle=True)
ridge_second_scores = cross_val_score(ridgeregr, X_train, Y_train, cv = kf_shuffle)


print("Mean of scores:", np.mean(ridge_scores))
print("STD of scores:", np.std(ridge_scores))      
print("\n")
print("Mean of second scores:", np.mean(ridge_second_scores))
print("STD of second scores:", np.std(ridge_second_scores)) 


Mean of scores: 0.782221643811832
STD of scores: 0.07061930436644727


Mean of second scores: 0.7704445091285447
STD of second scores: 0.12567516895131622


## Analysis of Added Features

After adding interest rate to the dataframe I ran a correlation matrix to see if there was any connection between the interest rate and sales price. Shocker- there wasn't. Interest rate isn't really something I personally think of as a big factor in whether to buy a house or not, so I am not surprised that it was not really related to sales price. Despite this low correlation I decided to try it in my model, just for kicks. According to the statistics after adding it to the model it really didn't help improve the accuracy of the model, so, if I was going forward with this model, I would not include interest rate as a feature in my X-variables. 

Adding additional information from outside the dataset is a great idea, but, in practice, it has a ton of issues. One of them is finding the data to go with what we want to know and making sure it actually is accurate. I would love to take the U.S. Census demographic information from this time period for this area and see if the data connects, but at my current skill level and time limitations I do not feel I could successfully do this within the time alotted for the assignment. So, in summary, extra information is always good but it is worth taking the time to gather it correctly and double check for its accuracy before adding it to your model.